In [ ]:
from langchain_google_genai import GoogleGenerativeAI
import time

api_key = "1222233"
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=api_key, temperature=.7)

In [20]:
res = llm.invoke("Write a 4-line poem about my love for samosas")
time.sleep(.5)
print(res)

Crispy, flaky, filled with delight,
Samosas, a savory bite.
Golden triangles, a street food's dream,
My love for you, a flavorsome gleam.


In [21]:
essay = llm.invoke("Write email requesting refund for electronic item")
time.sleep(.5)
print(essay)

Dear [Customer Service Representative Name],

I am writing to request a refund for an electronic item I recently purchased from your store.

On [Date], I purchased a [Product Name] (Order Number: [Order Number]) for [Price]. Unfortunately, upon receiving the item, I realized that it was not what I had expected.

The item I received does not meet my specifications for [Reason for Return]. I have contacted your customer service team via phone but was unable to resolve the issue.

I have attached a copy of my receipt for your reference. I would like to request a full refund for the item. I am happy to return the product to your store or arrange for pickup.

Please let me know the necessary steps to process my refund. I would appreciate a prompt response and resolution to this matter.

Thank you for your attention to this request.

Sincerely,
[Your Name]


In [22]:
from langchain.document_loaders.csv_loader import CSVLoader
# from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader('codebasics_faqs.csv', source_column='prompt')
data = loader.load()

In [23]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

HFEmbeddings = HuggingFaceEmbeddings()

vectordb = FAISS.from_documents(documents=data, embedding=HFEmbeddings)

In [24]:
retriever = vectordb.as_retriever()
rdocs = retriever.get_relevant_documents("any offer is there?")
rdocs

[Document(metadata={'source': 'Do you provide any virtual internship?', 'row': 14}, page_content='prompt: Do you provide any virtual internship?\nresponse: Yes'),
 Document(metadata={'source': 'Will this course guarantee me a job?', 'row': 33}, page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.'),
 Document(metadata={'source': 'I am encountering an error with the pre-invoice discount where I am receiving "0" values. Could you a

In [31]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


In [33]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs={"prompt":PROMPT}
)

In [38]:
chain("I have windows Computer. can i use powerbi on it?")

{'query': 'I have windows Computer. can i use powerbi on it?',
 'result': "I don't know.",
 'source_documents': [Document(metadata={'source': 'How can I use PowerBI on my Mac system?', 'row': 44}, page_content='prompt: How can I use PowerBI on my Mac system?\nresponse: Hi\n\nYou can use VirtualBox to create a virtual machine and install Windows on it. This will allow you to run Power BI and Excel on your Mac.\n\nIf you\'re not familiar with setting up a virtual machine, there are many resources available on YouTube that can guide you through the process. Simply search for "installing virtual machines" and you\'ll find plenty of helpful videos.\n\nBest of luck with your studies!'),
  Document(metadata={'source': 'Does Power BI work in Mac OS/Ubuntu?', 'row': 31}, page_content='prompt: Does Power BI work in Mac OS/Ubuntu?\nresponse: Power BI desktop works only in Windows OS. Please look into the system requirements section on this page. However, you can use a virtual machine to install a